In [2]:
# we can convert the rules into before, after dictionaries
# then also convert each update into a list format
# iterate over each element of each update
# then check the two sides of the list from the element with the rule dictionaries

def parse_input(input_str: str):
    rules_str, updates = input_str.split('\n\n')
    # parse updates into list format
    update_list = []
    for update_str in updates.split('\n'):
        update = []
        for page in update_str.split(','):
            update.append(int(page))
        update_list.append(update)

    # print(update_list)

    # parse rules into dictionaries
    
    # before_dict: keys must come before values
    before_dict = dict()
    # after_dict: keys come after values
    after_dict = dict()

    for line in rules_str.split('\n'):
        [before, after] = [int(page) for page in line.split('|')]
        if before in before_dict:
            before_dict[before].append(after)
        else:
            before_dict[before] = [after]
        if after in after_dict:
            after_dict[after].append(before)
        else:
            after_dict[after] = [before]

    # print(before_dict)
    # print(after_dict)
    
    return update_list, before_dict, after_dict

class Manual:
    def __init__(self, update_list: list, before_rules: dict, after_rules: dict):
        self.update_list = update_list
        self.before_rules = before_rules
        self.after_rules = after_rules

    def checkUpdate(self, update: list):
        for index, element in enumerate(update):
            # splice the list into two parts, before and after element
            if index > 0:
                pre = update[:index] # list of numbers in the update set that come before the current num 
                # hence, none of these same numbers should be in the corresponding before_dict value
                for item in pre:
                    if item in self.before_rules.get(element, []):
                        return False
        
            if index < len(update)-1:
                post = update[index+1:]
                for item in post:
                    if item in self.after_rules.get(element, []):
                        return False

        return True
    
    def getMiddleNumber(self, update: list):
        # assumption is that list is of odd number length
        middle_index = int(len(update) / 2)
        return update[middle_index]
    
    def part1(self):
        valid_updates = [update for update in self.update_list if self.checkUpdate(update)]
        sum_of_middle_nums = sum([self.getMiddleNumber(update) for update in valid_updates])
        return sum_of_middle_nums

In [16]:
with open('data/test/5.txt', 'r', encoding='utf-8') as f:
    input_str = f.read()

update_list, before_dict, after_dict = parse_input(input_str)
manual = Manual(update_list, before_dict, after_dict)
manual.part1()


[[75, 47, 61, 53, 29], [97, 61, 53, 29, 13], [75, 29, 13], [75, 97, 47, 61, 53], [61, 13, 29], [97, 13, 75, 29, 47]]
{47: [53, 13, 61, 29], 97: [13, 61, 47, 29, 53, 75], 75: [29, 53, 47, 61, 13], 61: [13, 53, 29], 29: [13], 53: [29, 13]}
{53: [47, 75, 61, 97], 13: [97, 61, 29, 47, 75, 53], 61: [97, 47, 75], 47: [97, 75], 29: [75, 97, 53, 61, 47], 75: [97]}


143

In [17]:
with open('data/input/5.txt', 'r', encoding='utf-8') as f:
    input_str = f.read()

update_list, before_dict, after_dict = parse_input(input_str)
manual = Manual(update_list, before_dict, after_dict)
manual.part1()

[[52, 77, 83, 31, 94, 75, 34, 95, 29, 38, 82, 19, 41, 39, 27, 98, 84, 13, 55, 21, 66], [76, 17, 43, 93, 46, 67, 68, 28, 18, 32, 87, 15, 89, 27, 31], [28, 63, 89, 27, 13, 66, 98, 95, 19, 36, 55], [38, 21, 39, 86, 16, 62, 76, 85, 47, 35, 93, 67, 68, 28, 18], [46, 67, 68, 28, 18, 87, 15, 89, 27, 31, 52, 13, 75, 66, 29, 98, 95, 19, 44, 77, 84], [41, 34, 98, 29, 95, 96, 55, 86, 52, 13, 16, 38, 82], [55, 83, 21, 16, 62, 76, 56, 43, 46], [37, 14, 56, 17, 18, 16, 35, 67, 21, 38, 59, 85, 93], [63, 18, 55, 27, 98, 32, 84, 19, 89, 87, 52, 44, 31, 95, 77, 36, 66, 13, 28, 29, 15, 94, 75], [55, 76, 39, 16, 82, 19, 34, 66, 29, 96, 75], [77, 34, 84, 94, 36, 55, 83, 82, 41, 38, 21, 39, 96, 86, 16, 62, 76, 56, 85, 59, 47, 35, 37], [62, 17, 47, 39, 37, 86, 85, 74, 38, 14, 93, 16, 67], [37, 67, 56, 46, 85, 62, 43, 68, 96, 17, 86, 39, 14, 76, 41, 82, 16], [89, 44, 67, 98, 28, 13, 75, 31, 63, 87, 52, 84, 34, 68, 77, 32, 46], [41, 38, 39, 96, 62, 17, 56, 47, 35, 37, 93], [44, 84, 94, 36, 21, 16, 62, 76, 85, 

5091

In [34]:
# part 2: get only incorrectly ordered updates and fix them 
# need to fix the entries
# and then get the middle number of the fixed entry

# to fix entries, we just need to check the after rules and iterate from left to right
# we can use two stacks to 'sort' the numbers
# add all the elements of the list to the second stack 
# push the first element of the second stack onto the first stack
# from the second element onwards, withhold the number and check if prior numbers in the first stack are ok
# push numbers from the first stack back to second stack until sequence is ok, then push the withheld number into the first stack
# continue this sequence until the second stack is empty

from collections import deque
from tqdm.notebook import tqdm

class Manual2(Manual):
    def fixUpdate(self, update: list):
        stack1 = deque()
        stack2 = deque()
        # add all elements of list to second stack
        for item in update:
            stack2.append(item)

        # push the first element of second stack to the first stack
        # this first element is now assumed to be the start of the list (for now)
        item = stack2.pop()
        stack1.append(item)

        while stack2:
            item = stack2.pop()
            # if this popped item must come before some items in stack1, we need to pop everything in stack1 until we can safely add the item to stack1
            # check the before dict to see if there are any items that must come after the popped item
            constraints = self.after_rules.get(item, [])
            for c in constraints:
                if stack1:
                    if c in stack1:
                        stack2.append(stack1.pop())
                else:
                    break
            # now we can safely push the withheld item to stack1
            stack1.append(item)

        return list(stack1)[::-1]
        

    def part2(self):
        invalid_updates = [update for update in self.update_list if not self.checkUpdate(update)]
        print(len(invalid_updates))
        fixed_updates = tqdm([self.fixUpdate(update) for update in invalid_updates])
        sum_of_middle_nums = sum([self.getMiddleNumber(update) for update in fixed_updates])
        return sum_of_middle_nums



In [35]:
with open('data/test/5.txt', 'r', encoding='utf-8') as f:
    input_str = f.read()

update_list, before_dict, after_dict = parse_input(input_str)
manual = Manual2(update_list, before_dict, after_dict)
manual.part2()

3


  0%|          | 0/3 [00:00<?, ?it/s]

123

In [ ]:
with open('data/input/5.txt', 'r', encoding='utf-8') as f:
    input_str = f.read()

update_list, before_dict, after_dict = parse_input(input_str)
manual = Manual2(update_list, before_dict, after_dict)
manual.part2()

86


  0%|          | 0/86 [00:00<?, ?it/s]Process SpawnPoolWorker-10:
Process SpawnPoolWorker-8:
Process SpawnPoolWorker-9:
Traceback (most recent call last):
  File "/Users/kieron/anaconda3/envs/aoc_24/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/kieron/anaconda3/envs/aoc_24/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/kieron/anaconda3/envs/aoc_24/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/kieron/anaconda3/envs/aoc_24/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'Manual2' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Users/kieron/anaconda3/envs/aoc_24/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/kieron/anaconda3/envs/aoc_24/lib/python3.10/mu

KeyboardInterrupt: 

In [ ]:
from multiprocess import Pool, cpu_count
from tqdm import tqdm

class Manual2(Manual):
    def fixUpdate(self, update: list):
        # Your existing `fixUpdate` implementation
        stack1 = deque()
        stack2 = deque()
        for item in update:
            stack2.append(item)
        item = stack2.pop()
        stack1.append(item)
        while stack2:
            item = stack2.pop()
            constraints = self.after_rules.get(item, [])
            for c in constraints:
                if stack1:
                    if c in stack1:
                        stack2.append(stack1.pop())
                else:
                    break
            stack1.append(item)
        return list(stack1)[::-1]

    def part2(self):
        invalid_updates = [update for update in self.update_list if not self.checkUpdate(update)]
        print(len(invalid_updates))

        # Limit the number of processes to avoid overloading
        num_processes = min(cpu_count(), 4)  # Use up to 4 processes
        with Pool(num_processes) as pool:
            fixed_updates = list(tqdm(pool.imap(self.fixUpdate, invalid_updates), total=len(invalid_updates)))

        sum_of_middle_nums = sum([self.getMiddleNumber(update) for update in fixed_updates])
        return sum_of_middle_nums

if __name__ == '__main__':
    # Initialize the manual object and call `part2`
    with open('data/input/5.txt', 'r', encoding='utf-8') as f:
        input_str = f.read()

    update_list, before_dict, after_dict = parse_input(input_str)
    manual = Manual2(update_list, before_dict, after_dict)
    manual.part2()


86


  0%|          | 0/86 [00:00<?, ?it/s]Process SpawnPoolWorker-101:
Process SpawnPoolWorker-98:
Process SpawnPoolWorker-100:
Process SpawnPoolWorker-99:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/kieron/anaconda3/envs/aoc_24/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/kieron/anaconda3/envs/aoc_24/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/kieron/anaconda3/envs/aoc_24/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/kieron/anaconda3/envs/aoc_24/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/kieron/anaconda3/envs/aoc_24/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
Traceback (most recent call last):
AttributeError: Can't get attribute 'Manual2' on <module '__main__' (built

KeyboardInterrupt: 